### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_YrQssLZVr2nRCzaMaQhzgQDthuxk

 


In [4]:
ace.check_session_timeout(s)

14399.317144

## Alpha Mixing Templates

## Read all passed alphas

In [5]:
os.chdir('Results/signal')
file_direc = glob.glob('*')
file_direc

['A_signalform.csv',
 'B_signalform.csv',
 'C_signalform.csv',
 'D_signalform.csv',
 'pre_signalform.csv',
 'Sep03GLB_signalform.csv']

In [6]:
expressions = {}
for file in file_direc:
    expressions[file.split('_')[0]] = pd.read_csv(file)['expression']
#expressions

In [7]:
os.chdir('../..')

In [8]:
expressions.keys()

dict_keys(['A', 'B', 'C', 'D', 'pre', 'Sep03GLB'])

In [9]:
expressions['pre']

0    sig_pre_0 = group_zscore(ts_zscore(ts_backfill...
Name: expression, dtype: object

## Template A & pre mixing
use top 32 results

In [10]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [11]:
B = expressions['B']
pre = expressions['pre']
#Sep03GLB = expressions['Sep03GLB']

In [12]:
alpha_list_Bpre = []

for i in range(30):
    alpha_list_Bpre += [ace.generate_alpha(f'{B[i]};{pre[0]};add(scale_down(signal_B_{i}), scale_down(signal_pre_{0}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = x, truncation = 0.01, decay = 5) for x in ['COUNTRY', 'SECTOR']]

alpha_list_Bpre

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 5,
   'neutralization': 'COUNTRY',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'entry_B_0 = ts_rank( ts_std_dev(oth460_3l_vlc, 21), 252 ); maintenance_B_0 = -ts_regression(rsk70_mfm2_gemtrd_srtindcnt, ts_delay(rsk70_mfm2_gemtrd_anlystsn, 1), 252); quit_B_0 = entry_B_0 < 0.5;signal_B_0 =  trade_when(entry_B_0>0.55, maintenance_B_0, quit_B_0);sig_pre_0 = group_zscore(ts_zscore(ts_backfill(anl69_epss_best_eeps_cur_yr, 21), 21), industry);factor_pre_0 = group_zscore(ts_ir(returns, 252), industry);value_pre_0 = floor(rank(fnd28_value_05480/close)*4.99);signal_pre_0 = group_neutralize(sig_pre_0, value_pre_0 *factor_pre_0);add(scale_down(signal_B_0), scale_down(signal_pre_0), filter = false)'},
 {'type':

In [13]:
result = ace.simulate_alpha_list_multi(s, alpha_list_Bpre)

100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:09:41<00:00, 209.09s/it]


In [14]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,...,self_correlation,prod_correlation,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,25815099,20000000,647,611,0.5515,0.2493,0.0533,0.000904,2.65,3.94,...,NaN,NaN,entry_B_8 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,20773093,20000000,713,662,0.5521,0.2006,0.0279,0.000727,2.64,4.38,...,NaN,NaN,entry_B_16 = ts_rank( ts_std_dev(oth460_1l_ccn...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,25468835,20000000,634,599,0.5537,0.2459,0.0511,0.000888,2.62,3.93,...,NaN,NaN,entry_B_3 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,20569680,20000000,698,648,0.5541,0.1986,0.0246,0.000717,2.62,4.37,...,0.6243,0.6243,entry_B_14 = ts_rank( ts_std_dev(oth460_1l_ccn...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,19814618,20000000,629,596,0.5500,0.1913,0.0303,0.000696,2.53,4.29,...,0.6262,0.6262,entry_B_6 = ts_rank( ts_std_dev(oth460_1l_2dls...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
5,25226900,20000000,716,661,0.5473,0.2436,0.0477,0.000890,2.52,3.78,...,NaN,NaN,entry_B_16 = ts_rank( ts_std_dev(oth460_1l_ccn...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
6,19714200,20000000,641,607,0.5482,0.1904,0.0306,0.000695,2.52,4.27,...,NaN,NaN,entry_B_10 = ts_rank( ts_std_dev(oth460_1l_2dl...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
7,24924480,20000000,702,647,0.5498,0.2407,0.0449,0.000875,2.51,3.80,...,NaN,NaN,entry_B_14 = ts_rank( ts_std_dev(oth460_1l_ccn...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
8,19826093,20000000,644,611,0.5560,0.1914,0.0347,0.000689,2.50,4.26,...,0.5954,0.5954,entry_B_8 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
9,19577723,20000000,633,600,0.5578,0.1890,0.0335,0.000678,2.46,4.22,...,NaN,NaN,entry_B_3 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS


In [15]:
path = 'Results/mixing'
result_st1.to_csv(path + '/B&pre_mixing_add_3.csv')

In [16]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,date,endDate,limit,name,result,startDate,themes,value,year
0,eOXZlZz,NaN,NaN,1.58,LOW_SHARPE,PASS,NaN,NaN,4.2200,NaN
1,eOXZlZz,NaN,NaN,1.00,LOW_FITNESS,PASS,NaN,NaN,2.4600,NaN
2,eOXZlZz,NaN,NaN,0.01,LOW_TURNOVER,PASS,NaN,NaN,0.5578,NaN
3,eOXZlZz,NaN,NaN,0.70,HIGH_TURNOVER,PASS,NaN,NaN,0.5578,NaN
4,eOXZlZz,NaN,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [17]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['QawpmEQ' 'jZqKNg5' 'k0xVv8k' 'R8RP661' 'n2prAQM' 'QVP6vA5' 'zmqe3nK'
 'P30KomK' 'plw3kRb' 'WGjJReZ' 'QVP6AYW' 'QVPoPJg' 'EE525EL' 'MQP2oZo'
 'plwpApj']
Passed alphas:['7weR901', 'anrLM6W', 'VPke0dM', 'xkA51lw', 'R8RvRMd', '6w5Ln6E', '3RM9oKZ', '9rqkEdK', 'KbWMjRx', 'e0l95Zp', '2L1O7wZ', 'anrLeKv', 'm8EWMk1', 'llL8EAn', 'JOVxpPm', 'olmkM05', 'j02M2zE', 'k3z6qqk', 'R8JVmRd', 'qAQJMKA', 'Lp0YPva', 'qlMwMj2', '8zMx1kW', 'Ppkmde7', 'APR0O6W', 'gMYQaXQ', '273obEY', 'lVoPLnn', 'Y0j5nrW', 'Wpv2EZo', 'xxJpKLg', 'QawpLe5', 'K9KOwjx', '17rAQ6m', 'K9KObrj', '9wNOzNr', 'Nr121bg', 'QV135m5', 'gMYQeNv', 'mbPg9gK', 'eOXZlZz', 'ad59v2O', '9rzVpZd', 'qlKWv9P', 'ZnR0E53']


In [18]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep10_mixing'])

In [21]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas[12:14]}
submit_result

{'m8EWMk1': True, 'llL8EAn': False}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
